In [2]:
import os
from utils import *
import optuna
from Recommenders.Recommender_import_list import *

Tensorflow is not available


c:\Users\Francesco1\miniconda3\envs\RecSysFramework\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [3]:
data, usermap, itemmap, users = load_data2()
data_train, data_val = split_data2(data, 0.2)

In [4]:
candidate_recommender_study = optuna.create_study(
    study_name="P3Beta",
    #storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)
candidate_recommender = RP3betaRecommender(data_train, verbose=False)
candidate_recommender.fit()

[I 2023-12-29 17:37:21,387] A new study created in memory with name: P3Beta


In [5]:
n_users, n_items = data_train.shape

training_dataframe = pd.DataFrame(index=range(0,n_users), columns = ["ItemID"])
training_dataframe.index.name='UserID'

In [6]:
cutoff = 50

for user_id in range(n_users):    
    recommendations = candidate_recommender.recommend(user_id, cutoff = cutoff)
    training_dataframe.loc[user_id, "ItemID"] = recommendations
training_dataframe = training_dataframe.explode("ItemID")

In [7]:
import scipy.sparse as sps

data_val_coo = sps.coo_matrix(data_val)
correct_recommendations = pd.DataFrame({"UserID": data_val_coo.row,
                                        "ItemID": data_val_coo.col})
training_dataframe = pd.merge(training_dataframe, correct_recommendations, on=['UserID','ItemID'], how='left', indicator='Exist')
training_dataframe["Label"] = training_dataframe["Exist"] == "both"
training_dataframe.drop(columns = ['Exist'], inplace=True)

In [8]:
topPop = TopPop(data_train)
topPop.fit()

p3alpha_study = optuna.create_study(
    study_name="P3Alpha",
    #storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)
p3alpha = P3alphaRecommender(data_train)
p3alpha.fit()

#easer_study = optuna.create_study(
#    study_name="Easer",
#    #storage=get_database_url(),
#    load_if_exists=True,
#    direction="maximize",
#)
#easer = EASE_R_Recommender(data_train)
#easer.fit(topK=72, l2_norm=36.47305040353163)

itemknncf_study = optuna.create_study(
    study_name="ItemKNNCF",
    #storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)
itemknncf = ItemKNNCFRecommender(data_train)
itemknncf.fit()


other_algorithms = {
    "TopPop": topPop,
    "P3alpha": p3alpha,
    #"Easer": easer,
    "ItemKNNCF": itemknncf
}

[I 2023-12-29 17:37:37,735] A new study created in memory with name: P3Alpha


TopPopRecommender: URM Detected 220 ( 1.7%) users with no interactions.
TopPopRecommender: URM Detected 97 ( 0.4%) items with no interactions.
P3alphaRecommender: URM Detected 220 ( 1.7%) users with no interactions.
P3alphaRecommender: URM Detected 97 ( 0.4%) items with no interactions.
P3alphaRecommender: Similarity column 22222 (100.0%), 3175.01 column/sec. Elapsed time 7.00 sec


[I 2023-12-29 17:37:46,308] A new study created in memory with name: ItemKNNCF


ItemKNNCFRecommender: URM Detected 220 ( 1.7%) users with no interactions.
ItemKNNCFRecommender: URM Detected 97 ( 0.4%) items with no interactions.
Similarity column 22222 (100.0%), 8723.61 column/sec. Elapsed time 2.55 sec


In [9]:
training_dataframe = training_dataframe.set_index('UserID')

for user_id in range(n_users):  
    for rec_label, rec_instance in other_algorithms.items():
        item_list = training_dataframe.loc[user_id, "ItemID"].values.tolist()
        all_item_scores = rec_instance._compute_item_score([user_id], items_to_compute = item_list)
        training_dataframe.loc[user_id, rec_label] = all_item_scores[0, item_list] 

training_dataframe = training_dataframe.reset_index()
training_dataframe = training_dataframe.rename(columns = {"index": "UserID"})

item_popularity = np.ediff1d(sps.csc_matrix(data_train).indptr)
training_dataframe['item_popularity'] = item_popularity[training_dataframe["ItemID"].values.astype(int)]
user_popularity = np.ediff1d(sps.csr_matrix(data_train).indptr)
training_dataframe['user_profile_len'] = user_popularity[training_dataframe["UserID"].values.astype(int)]

In [10]:
training_dataframe = training_dataframe.set_index('ItemID')
training_dataframe = training_dataframe.reset_index()
training_dataframe = training_dataframe.rename(columns = {"index": "ItemID"})

In [11]:
groups = training_dataframe.groupby("UserID").size().values
train_recs = training_dataframe.drop(columns=["Label"])
train_labs = training_dataframe["Label"]

In [12]:
study = optuna.create_study(
    study_name="XGBoost(P3Beta, TopPop, P3alpha, Easer, ItemKNNCF)",
    #storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)

[I 2023-12-29 17:38:08,204] A new study created in memory with name: XGBoost(P3Beta, TopPop, P3alpha, Easer, ItemKNNCF)


In [13]:
def evaluator(data_train, train_recs, XGB_model, training_dataframe, data_test: sp.csr_matrix, recommendation_length: int = 10):
    accum_precision = 0
    accum_recall = 0
    accum_map = 0

    num_users = data_train.shape[0]

    num_users_evaluated = 0
    num_users_skipped = 0
    for user_id in range(num_users):
        user_profile_start = data_test.indptr[user_id]
        user_profile_end = data_test.indptr[user_id+1]

        relevant_items = data_test.indices[user_profile_start:user_profile_end]

        if relevant_items.size == 0:
            num_users_skipped += 1
            continue
        
        predict = train_recs[train_recs["UserID"] == user_id]
        predictions = XGB_model.predict(predict)
        user_ids = predict["UserID"].values
        item_ids = predict["ItemID"].values
        predictions = pd.DataFrame({"UserID": user_ids, "ItemID": item_ids, "Predictions": predictions})
        predictions = predictions.sort_values(by="Predictions", ascending=False)
        recommendations = np.array(predictions["ItemID"].values[:recommendation_length])

        accum_precision += precision(recommendations, relevant_items)
        accum_recall += recall(recommendations, relevant_items)
        accum_map += mean_average_precision(recommendations, relevant_items)

        num_users_evaluated += 1


    accum_precision /= max(num_users_evaluated, 1)
    accum_recall /= max(num_users_evaluated, 1)
    accum_map /=  max(num_users_evaluated, 1)

    return accum_precision, accum_recall, accum_map, num_users_evaluated, num_users_skipped


In [16]:
from xgboost import XGBRanker

def objective(trial):
    objective = trial.suggest_categorical("objective", ["rank:ndcg", "rank:map", "rank:pairwise"])
    n_estimators = trial.suggest_int("n_estimators", 5, 500)
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-1, log=True)
    reg_alpha = trial.suggest_float("reg_alpha", 1e-4, 1e-1, log=True)
    reg_lambda = trial.suggest_float("reg_lambda", 1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int("max_depth", 1, 10)
    max_leaves = trial.suggest_int("max_leaves", 1, 10)
    grow_policy = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
    booster = trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"])

    XGB_model = XGBRanker(
        tree_method="hist",
        objective=objective,
        n_estimators=int(n_estimators),
        random_state=None,
        learning_rate=learning_rate,
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda,
        max_depth=int(max_depth),
        max_leaves=int(max_leaves),
        grow_policy=grow_policy,
        booster=booster,
        verbosity=0, # 2 if self.verbose else 0,
    )
    XGB_model.fit(
        train_recs,
        train_labs,
        group=groups,
        verbose=False
    )
    _, _, ev_map, _, _ = evaluator(data_train, train_recs, XGB_model, training_dataframe, data_val)
    
    return ev_map

study.optimize(objective, n_trials=2)

[I 2023-12-29 18:08:44,535] Trial 4 finished with value: 0.043258269653554615 and parameters: {'objective': 'rank:map', 'n_estimators': 489, 'learning_rate': 0.018590811418379614, 'reg_alpha': 0.003700046939843544, 'reg_lambda': 0.045807529399430995, 'max_depth': 2, 'max_leaves': 9, 'grow_policy': 'depthwise', 'booster': 'gblinear'}. Best is trial 2 with value: 0.05127384661844544.
[I 2023-12-29 18:10:22,338] Trial 5 finished with value: 0.0682245791815283 and parameters: {'objective': 'rank:map', 'n_estimators': 70, 'learning_rate': 0.0008765542961329935, 'reg_alpha': 0.012021960865902171, 'reg_lambda': 0.0004814250009694544, 'max_depth': 4, 'max_leaves': 7, 'grow_policy': 'lossguide', 'booster': 'dart'}. Best is trial 5 with value: 0.0682245791815283.


In [17]:
study.best_params

{'objective': 'rank:map',
 'n_estimators': 70,
 'learning_rate': 0.0008765542961329935,
 'reg_alpha': 0.012021960865902171,
 'reg_lambda': 0.0004814250009694544,
 'max_depth': 4,
 'max_leaves': 7,
 'grow_policy': 'lossguide',
 'booster': 'dart'}

In [ ]:
# Stampare solo le righe con almeno un NaN
for row_index in data_val:
    if np.isnan(row_index.data).any():
        print(row_index)

In [1]:
from xgboost import XGBRanker
import pandas as pd

#data_train = data_train.fillna(0)  # O sostituisci con un valore appropriato
#data_val = data_val.fillna(0)

recommender = XGBRanker(XGB_model, verbose=False)
recommender.fit(**study.best_params)

NameError: name 'XGB_model' is not defined

In [ ]:
def submission2(recommender, users, usermap, itemmap, data_train):
    
    toppoprecommender = TopPop(data_train)
    toppoprecommender.fit()
    
    with open('data/results.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['user_id', 'item_list'])
        for user_id in users:
            try:
                item_list = recommender.recommend(user_id, cutoff=10)
                item_list = [itemmap[i] for i in item_list]
                writer.writerow([usermap[user_id], ' '.join(map(str, item_list))])
            except IndexError:
                item_list = toppoprecommender.recommend(0, cutoff=10)
                item_list = [itemmap[i] for i in item_list]
                writer.writerow([usermap[user_id], ' '.join(map(str, item_list))])


In [ ]:
submission2(recommender, users, usermap, itemmap, data_train)

NameError: name 'recommender' is not defined